In [106]:
import pandas as pd
import clr
import configparser

path = "data/sample/test.csv"

config = configparser.ConfigParser()
config.read('config\settings.ini')

newtonsoftjson_path = config["Paths"]["Newtonsoft.Json.Dll"]
wsconnector_path = config["Paths"]["WSConnector.Dll"]

clr.AddReference(newtonsoftjson_path)
clr.AddReference(wsconnector_path)

from WSConnector import Connector

connector = Connector()


In [107]:
# read the CSV file and store it in a DataFrame
df = pd.read_csv(path, header=4)

# add a new column "Measurement" and initialize it with NaN values
df["Measurement"] = ""
df["PartNumber"] = ""
df["Station"] = "STA_FNLFUNC172"
# loop through each row of the DataFrame
for i, row in df.iterrows():
    if row["Status"] == "Failed":
        measurement = row["FirstFail"]
        # search for the measurement in the header
        measurement_column = df.columns.get_loc(measurement)
        # write the value of the measurement in the new column "Measurement"
        df.at[i, "Measurement"] = df.iloc[i, measurement_column]
    serial = row["SN"]
    resp, serial_partnumber = connector.CIMP_PartNumberRef(serial,1, serial_partnumber)
    df.at[i, "PartNumber"] = serial_partnumber

# select the desired columns
df['Only_Date'] = df['Date'].str.split(" ").str[0]
df = df[["Station", "SN", "PartNumber", 'Date', "Only_Date", "CycleTime", "Status", "FirstFail", "Measurement"]]

df

,Station,SN,PartNumber,Date,Only_Date,CycleTime,Status,FirstFail,Measurement
0,STA_FNLFUNC172,S224800010,X13650728060+M-KEM,23/01/2023 02:57:31 p. m.,23/01/2023,198.66s,Failed,UUT Current Measurment,0.183029
1,STA_FNLFUNC172,S235405749,,23/01/2023 03:01:49 p. m.,23/01/2023,192.42s,Failed,1.35 VDC Measurment,1.35107
2,STA_FNLFUNC172,S230600038,X13650728070+M-KMX,23/01/2023 03:05:25 p. m.,23/01/2023,192.60s,Passed,NaN,
3,STA_FNLFUNC172,S214600912,X13651736001+C,23/01/2023 03:18:01 p. m.,23/01/2023,198.97s,Passed,NaN,
4,STA_FNLFUNC172,S214600912,X13651736001+C,23/01/2023 03:21:47 p. m.,23/01/2023,198.96s,Failed,VDC w/7 VAC Power,1.564014
5,STA_FNLFUNC172,S230803894,,23/01/2023 03:26:07 p. m.,23/01/2023,189.78s,Failed,VDC w/18 VAC Power,23.79631


In [130]:
# header_df = {'Station': [], 'SN': [],'PartNumber': [], 'Date': [], 'Only_Date': [], 'CycleTime': [], 'Status': [], 'FirstFail': [], 'Measurement': []}
# main_df = pd.DataFrame(header_df)

main_df = main_df.append(df, ignore_index= True)

main_df

# main_df = pd.DataFrame()

# main_df

C:\Users\K90011729\AppData\Local\Temp\ipykernel_15472\4006056597.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.append(df, ignore_index= True)


In [129]:
# save the new DataFrame to a new CSV file
main_df.to_csv("data/sample/output.csv", index=False)